<span id="test-locally"></span>
# Qiskit Runtime local testing mode

Local testing mode (available with `qiskit-ibm-runtime` 0.22.0 or later) can be used to help develop and test programs before fine-tuning them and sending them to real quantum hardware.  After using local testing mode to verify your program, all you need to change is the backend name to run it on an IBM Quantum system.

To use local testing mode, specify one of the fake backends from ``qiskit_ibm_runtime.fake_provider`` or specify a Qiskit Aer backend when instantiating a Qiskit Runtime primitive or a session.

- **Fake backends**: The [fake backends](/api/qiskit-ibm-runtime/fake_provider)  in ``qiskit_ibm_runtime.fake_provider`` mimic the behaviors of IBM Quantum&trade; systems by using system snapshots. The system snapshots contain important information about the quantum system, such as the coupling map, basis gates, and qubit properties, which are useful for testing the transpiler and performing noisy simulations of the system. The noise model from the snapshot is automatically applied during simulation.
- **Aer simulator**: Simulators from [Qiskit Aer](simulate-with-qiskit-aer) provide higher-performance simulation that can handle larger circuits and [custom noise models](./build-noise-models). It also supports Clifford simulation mode, which can efficiently simulate Clifford circuits with a large number of qubits.


<Admonition type="note" title="Notes">
You can specify all Qiskit Runtime options in local testing mode.  However, all options except shots are ignored when run on a local simulator.
</Admonition>


{/* If you currently use the Qiskit SDK, you can use the existing Qiskit Aer primitives for local testing.  See [exact and noisy simulation with Qiskit Aer primitives](./simulate-with-qiskit-aer) for details. */}

## Fake backends example

In [1]:
from qiskit.circuit import QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit_ibm_runtime.fake_provider import FakeManilaV2

# Bell Circuit
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

# Run the sampler job locally using FakeManilaV2
fake_manila = FakeManilaV2()
pm = generate_preset_pass_manager(backend=fake_manila, optimization_level=1)
isa_qc = pm.run(qc)

# You can use a fixed seed to get fixed results.
options = {"simulator": {"seed_simulator": 42}}
sampler = Sampler(mode=fake_manila, options=options)

result = sampler.run([isa_qc]).result()

## AerSimulator examples

Example with sessions, without noise:

In [2]:
from qiskit_aer import AerSimulator
from qiskit.circuit import QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import Session, SamplerV2 as Sampler

# Bell Circuit
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

# Run the sampler job locally using AerSimulator.
# Session syntax is supported but ignored because local mode doesn't support sessions.
aer_sim = AerSimulator()
pm = generate_preset_pass_manager(backend=aer_sim, optimization_level=1)
isa_qc = pm.run(qc)
with Session(backend=aer_sim) as session:
    sampler = Sampler(mode=session)
    result = sampler.run([isa_qc]).result()

To simulate with noise, specify a system (quantum hardware) and submit it to Aer.  Aer builds a noise model based on the calibration data from that system, and instantiates an Aer backend with that model.  If you prefer, you can [build a noise model](./build-noise-models).

Example with noise:

In [3]:
from qiskit_aer import AerSimulator
from qiskit.circuit import QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import SamplerV2 as Sampler, QiskitRuntimeService

# Bell Circuit
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

service = QiskitRuntimeService()

# Specify a system to use for the noise model
real_backend = service.backend("ibm_brisbane")
aer = AerSimulator.from_backend(real_backend)

# Run the sampler job locally using AerSimulator.
pm = generate_preset_pass_manager(backend=aer, optimization_level=1)
isa_qc = pm.run(qc)
sampler = Sampler(mode=aer)
result = sampler.run([isa_qc]).result()

<span id="clifford-sim"></span>
### Clifford simulation

Because Clifford circuits can be simulated efficiently with verifiable results, Clifford simulation is a very useful tool. For an in-depth example, see [Efficient simulation of stabilizer circuits with Qiskit Aer primitives](./simulate-stabilizer-circuits).

Example:

In [4]:
import numpy as np
from qiskit.circuit.library import EfficientSU2
from qiskit_ibm_runtime import SamplerV2 as Sampler

n_qubits = 500  # <---- note this uses 500 qubits!
circuit = EfficientSU2(n_qubits)
circuit.measure_all()

rng = np.random.default_rng(1234)
params = rng.choice(
    [0, np.pi / 2, np.pi, 3 * np.pi / 2],
    size=circuit.num_parameters,
)

# Tell Aer to use the stabilizer (clifford) simulation method
aer_sim = AerSimulator(method="stabilizer")

pm = generate_preset_pass_manager(backend=aer_sim, optimization_level=1)
isa_qc = pm.run(qc)
sampler = Sampler(mode=aer_sim)
result = sampler.run([isa_qc]).result()

## Next steps

<Admonition type="tip" title="Recommendations">
    - Review detailed [primitives examples.](./primitives-examples)
    - Read [Migrate to V2 primitives](/api/migration-guides/v2-primitives).
    - Practice with primitives by working through the [Cost function lesson](https://learning.quantum.ibm.com/course/variational-algorithm-design/cost-functions#primitives) in IBM Quantum Learning.
    - Learn how to transpile locally in the [Transpile](./transpile) section.
    - Try the [Submit pre-transpiled circuits](https://learning.quantum.ibm.com/tutorial/submitting-user-transpiled-circuits-using-primitives) tutorial.
</Admonition>